In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, warnings
import gradio 
from datasets import load_dataset
from trl import SFTTrainer

/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
 print('메모리 사용량/총 메모리: ' + str(torch.cuda.memory_allocated('cuda:0')/1024**3 ) + 'GB / ' + str(torch.cuda.memory_reserved('cuda:0')/1024**3 ) + 'GB')

메모리 사용량/총 메모리: 0.0GB / 0.0GB


In [4]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: NVIDIA RTX A6000
Compute Capability: (8, 6)
Total Memory: 51041271808 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.2.0-34-generic
Python Version: 3.9.12


In [5]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ········
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through g

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Minirecord/Mini_synatra_7b_02")

model = AutoModelForCausalLM.from_pretrained("Minirecord/Mini_synatra_7b_02")
model.to('cuda:0')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [7]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    B_INST, E_INST = "### Instruction:", "### Response:"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=1500)

    # Decode and print the generated response
    print(tokenizer.decode(outputs[0]))

In [8]:
stream("나 행복해")

/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/transformers/generation/utils.py:1517: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:나 행복해### Response:> 안녕하세요! 저는 인공지능 언어 모델이기 때문에 개인적인 감정이나 감정을 가지고 있지 않습니다. 하지만 여러분의 기분을 밝게 해드리기 위해 여기 있습니다. 무엇이든 자유롭게 공유하고 궁금한 점이 있으면 언제든지 문의해 주세요. 저는 여기 있어서 여러분을 돕고 즐거운 시간을 보내기 위해 여기 있습니다.</s>


In [9]:
stream("나 오늘 떡볶이 먹어서 기뻐")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:나 오늘 떡볶이 먹어서 기뻐### Response: 떡볶이는 맛있는 요리입니다! 떡볶이를 먹을 때 기분이 좋다는 것은 정상입니다. 떡볶이는 맛있고 즐거운 식사를 즐기는 것과 같습니다. 떡볶이를 먹을 때 기분이 좋다면 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이 좋다는 것입니다. 떡볶이를 먹을 때 기분이 좋다는 것은 떡볶이를 먹을 때 기분이